<a href="https://colab.research.google.com/github/MathewS424/PyTorch-Tutorial/blob/main/14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT MNIST IMAGES - VIDEO 14**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Convert MNIST IMAGE Files into a Tensor of 4 -Dimensions(no.of images, height, width, Colorr channel)
transform = transforms.ToTensor()

In [4]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 33513258.97it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 2227455.16it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10110228.06it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1063265.54it/s]


Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
pwd

'/content'

In [9]:
ls

sample_data/


In [10]:
cd ../

/


In [11]:
pwd

'/'

In [12]:
ls

bin@       cuda-keyring_1.0-1_all.deb  home/   libx32@                   opt/         run/   tmp/
boot/      datalab/                    lib@    media/                    proc/        sbin@  tools/
cnn_data/  dev/                        lib32@  mnt/                      python-apt/  srv/   usr/
content/   etc/                        lib64@  NGC-DL-CONTAINER-LICENSE  root/        sys/   var/


In [13]:
cd cnn_data

/cnn_data


In [14]:
ls

MNIST/


In [15]:
cd ../

/


In [16]:
ls

bin@       cuda-keyring_1.0-1_all.deb  home/   libx32@                   opt/         run/   tmp/
boot/      datalab/                    lib@    media/                    proc/        sbin@  tools/
cnn_data/  dev/                        lib32@  mnt/                      python-apt/  srv/   usr/
content/   etc/                        lib64@  NGC-DL-CONTAINER-LICENSE  root/        sys/   var/


In [17]:
cd content/

/content


In [18]:
pwd

'/content'

**CONVOLUTIONAL AND POOLING LAYERS - VIDEO 15**

In [19]:
# Create a small batch size for images ..that is 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [20]:
# Define Our CNN Model
# Describe convolutional layer and what  its doing (2 convolutional layers)
# This is just am example in the next video we'll build out the actual model
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 =  nn.Conv2d(6, 16, 3, 1)  # we have not set padding parameters so the images becomes 26 X 26 it was 28 x 28 .
# becaues the boundry of the MINIST images are unusefull so its skipped the data of that boundry part and become 26x26(check output of In [26]:x.shape)
# Check Video -15 08:40

In [22]:
# Grab 1 MNIST RECORD/IMAGE
for i, (X_train, y_train) in enumerate(train_data):
  break

In [24]:
X_train.shape

torch.Size([1, 28, 28])

In [25]:
x = X_train.view(1, 1, 28, 28)

In [26]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [27]:
# 1 images, 6 is the filter
x.shape   # How it becomes 26 x 26 anser is in In[20] or # Check Video -15 08:40

torch.Size([1, 6, 26, 26])

In [29]:
# pass through the pooling layer
x = F.max_pool2d(x, 2, 2) #kernel of 2 and stride of 2

In [30]:
x.shape  # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [31]:
# Do our second Convolution
x = F.relu(conv2(x))

In [32]:
x.shape  #  we did not set any padding in conv2 so the img pixel was 13 x 13 reduce 2 becomes 11 x 11
# (reduced around outside of the images)

torch.Size([1, 16, 11, 11])

In [33]:
# Do another pooling layer
x = F.max_pool2d(x, 2, 2)  #kernel of 2 and stride of 2

In [35]:
x.shape  # 11 / 2 = 5.5 but we have to round down, because you can't invent data to round up

torch.Size([1, 16, 5, 5])

In [37]:
((28-2) / 2 -2) / 2  # order of reducing images from convolutional layer ,pooling layer to 5.5 pixel from 28 pixel initially

5.5

**CONVOLUTIONAL NEURAL NETWOORK MODEL -VIDEO - 16**

In [40]:
# Model class - above code together
class ConvolutionalNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    # 1st convolutional layer
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    # 2nd convolutional layer
    self.conv2 = nn.Conv2d(6, 16, 3, 1)

    # Fully connected Layer after Conv ,pooling layers its fully connected layer check image of CNN
    self.fc1 = nn.Linear(5*5*16, 120)  # 120,84,10 are randoms
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)  # 2x2 kernel and stride of 2
    # Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    # Re-View to flattten it out
    X = X.view(-1, 5*5*16)  # -1 SO THAT WE CAN VARY THE BATCH SIZE

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    x = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [41]:
# Create an Instance of  our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [42]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # learning rate of 0.01 smaller lr ,longer its gonna take to train

**### Detailed Explanation of the Code Lines**

I'll explain each part of the code as if you're learning it for the first time, with a focus on what each line does, why certain parameters are used, and how the numbers fit into the overall picture.

#### [3] Convert MNIST IMAGE Files into a Tensor of 4-Dimensions

```python
transform = transforms.ToTensor()
```

- **What it does:** This line of code converts images into a tensor, which is a multi-dimensional array that PyTorch can process.
- **Why it's used:** The MNIST dataset images are originally in PIL format (Python Imaging Library), and we need to convert them into tensors so they can be used in deep learning models. Tensors allow the model to perform mathematical operations on the data.
- **4-Dimensions:** When we convert an image to a tensor, it becomes a 4D tensor of shape `(number of images, channels, height, width)`. For grayscale images like MNIST, the number of channels is 1.

#### [4] Train Data

```python
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)
```

- **What it does:** This line downloads the MNIST dataset and prepares it for training.
- **Parameters:**
  - `root='/cnn_data'`: Specifies the directory where the dataset will be stored.
  - `train=True`: Indicates that we are downloading the training set.
  - `download=True`: If the dataset isn't already in the specified directory, it will be downloaded.
  - `transform=transform`: Applies the transformation (conversion to tensor) to each image in the dataset.
  
#### [5] Test Data

```python
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)
```

- **What it does:** Similar to the training data, but this line downloads the test set.
- **Parameters:**
  - `train=False`: This specifies that we are downloading the test set.

#### [19] Create Data Loaders

```python
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)
```

- **What it does:** These lines create data loaders, which are tools that help load the data in batches for training and testing.
- **Parameters:**
  - `batch_size=10`: Specifies that each batch will contain 10 images. Smaller batches require less memory but take longer to train.
  - `shuffle=True`: Randomly shuffles the training data at every epoch to help the model generalize better.
  - `shuffle=False`: The test data is not shuffled because we want consistent results when evaluating the model.

#### [20] Define Convolutional Layers

```python
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)
```

- **What it does:** These lines define two convolutional layers in the neural network.
- **Parameters:**
  - `1`: The number of input channels. For MNIST, this is 1 because the images are grayscale.
  - `6` and `16`: The number of output channels (filters). Each filter will learn to detect different features in the image.
  - `3`: The size of the convolutional filter (3x3).
  - `1`: The stride, which indicates how the filter moves across the image. A stride of 1 means it moves 1 pixel at a time.

#### [22] Grab One Image from the Dataset

```python
for i, (X_train, y_train) in enumerate(train_data):
  break
```

- **What it does:** This code iterates over the dataset and breaks after getting the first image.
- **Why it's used:** We want to grab a single image to see what it looks like and to check its shape.

#### [24] Check Image Shape

```python
X_train.shape
```

- **What it does:** This line returns the shape of the image tensor.
- **Output:** For an MNIST image, this would be `[1, 28, 28]`, meaning 1 channel (grayscale), and 28x28 pixels in height and width.

#### [25] Reshape the Image

```python
x = X_train.view(1, 1, 28, 28)
```

- **What it does:** This reshapes the image to add an additional dimension, making it ready for the convolutional layer.
- **Parameters:**
  - `1, 1, 28, 28`: The shape of the tensor. The first `1` is the batch size, and the second `1` is the number of channels. The last two numbers are the height and width.

#### [26] Perform the First Convolution

```python
x = F.relu(conv1(x))
```

- **What it does:** Applies the first convolutional layer followed by a ReLU activation function.
- **Why ReLU:** ReLU (Rectified Linear Unit) introduces non-linearity to the model, allowing it to learn more complex patterns.

#### [27] Check Shape After Convolution

```python
x.shape
```

- **Output:** The shape is `[1, 6, 26, 26]`. The height and width reduce to 26x26 because the 3x3 filter reduces the image size by 2 pixels in each dimension.

#### [29] Apply Max Pooling

```python
x = F.max_pool2d(x, 2, 2)
```

- **What it does:** Applies a max pooling operation, which reduces the size of the image by taking the maximum value in each 2x2 region.
- **Parameters:**
  - `2, 2`: The size of the pooling filter and the stride. This reduces the image size by half.

#### [30] Check Shape After Pooling

```python
x.shape
```

- **Output:** The shape is `[1, 6, 13, 13]`, as the pooling operation reduces the dimensions by half.

#### [31] Perform the Second Convolution

```python
x = F.relu(conv2(x))
```

- **What it does:** Applies the second convolutional layer followed by ReLU activation.
- **Shape Reduction:** The output shape after this operation becomes `[1, 16, 11, 11]` because the filter reduces the dimensions further.

#### [33] Apply Second Max Pooling

```python
x = F.max_pool2d(x, 2, 2)
```

- **What it does:** Applies another max pooling operation to further reduce the size of the image.

#### [35] Check Final Shape

```python
x.shape
```

- **Output:** The final shape is `[1, 16, 5, 5]`, after all the convolution and pooling operations.

#### [40] Model Class

```python
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
```

- **What it does:** Defines a convolutional neural network (CNN) class with two convolutional layers.
- **Why Classes:** In PyTorch, models are usually defined as classes to organize layers and operations in a clean way.

#### [41] Create Model Instance

```python
torch.manual_seed(41)
model = ConvolutionalNetwork()
```

- **What it does:** Creates an instance of the model and sets a random seed for reproducibility.
- **Why Seed:** Setting a seed ensures that the results are reproducible, meaning you'll get the same results every time you run the code.

#### [42] Loss Function and Optimizer

```python
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
```

- **What it does:** Defines the loss function and the optimizer.
- **Parameters:**
  - `nn.CrossEntropyLoss()`: The loss function used for classification tasks, comparing the model's output with the true labels.
  - `torch.optim.Adam`: An optimizer that adjusts the model parameters to minimize the loss function.
  - `lr=0.001`: The learning rate, which controls how much to adjust the model parameters with each update. A smaller learning rate means more gradual learning.

### Summary
This code builds a basic Convolutional Neural Network (CNN) for the MNIST dataset. It includes data loading, model definition, and preparation for training by setting up the loss function and optimizer. Each step transforms the images and their dimensions, leading to the final structure that the model uses to make predictions.

**TRAIN AND TEST CNN MODEL - VIDEO 17**

In [44]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
train_correct = []
test_losses = []
test_correct = []

# For loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b +=1  # start our batches at 1
    y_pred = model(X_train)  # get predicted values from the training set.Not FLATTENED ITS 2d
    print(y_pred)
    loss = criterion(y_pred, y_train)  # compare the predicted


    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions.
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch.
    trn_corr += batch_corr  # keep track as we go along in training
    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out some results
    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Test



current_time = time.time()
total = current_time - start_time
print(f"Training Took: {total/60} minutes!")

1724419833.9779313